In [29]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [30]:
import tensorflow as tf
from skimage.metrics import structural_similarity as ssim
import os
import matplotlib.pyplot as plt
import statistics
import pickle

import math
import cv2
import numpy as np
from numpy import cos as cos
from numpy import sin as sin
from numpy import sqrt as sqrt
from numpy import arctan2 as arctan2
from matplotlib import pyplot as plt
import os
import datetime
import time
import matplotlib 

# Diff-PAT for Phase Plates
# Author: Tatsuki FUSHIMI, Kenta YAMAMOTO
# Corresponding Author: Tatsuki FUSHIMI (tfushimi@slis.tsukuba.ac.jp)
# Date: 2020/11/28

# Initialization function
def init_param():
    f0 = 2e06 # frequenc
    c0 = 1480 # speed of sound in water
    l_ambda = c0/f0 # wavelength
    N = 256 # image resolution
    p = 0.00015 # pixel size

    z = 0.02 # distance to the plane
    T0 = 3e-03 # thickness of plate (not used in the calculation)

    return N, z, p, l_ambda, T0, f0, c0

# Normlization function for output
def amp_abs(amp_cgh_img):
    new_amp_cgh = amp_cgh_img / amp_cgh_img.max() * 255.0
    return new_amp_cgh

# Normlization function for phase output
def phaser_draw(phase_dist):
    new_phase_dist = phase_dist.numpy()
    while new_phase_dist.min() < 0:
      new_phase_dist += math.pi*2
    new_phase_dist = (new_phase_dist % (2*math.pi))/(2*math.pi)
    new_phase_dist = new_phase_dist * 255
    return new_phase_dist

# Visualization Function
def show_imgs(imgs):
    for i in range(len(imgs)):
      img = imgs[i]
      plt.figure(figsize=(6,6))
      plt.imshow(img)
      plt.gray()
    plt.show()

# Transmission Loss calculation, not used in the manuscript
def calculate_alpha(phase, f0):
    # Hologram
    c_h = 2424
    rho_h = 1190
    Z_h = c_h * rho_h
    k_h = (2*math.pi*f0) / c_h
    
    #transducer surface (assume gel to be impedance matched with hologram)
    Z_t = Z_h
    # water
    rho_m = 1000
    c_m = 1480
    Z_m = c_m * rho_m
    k_m = (2*math.pi*f0) / c_m
    #Hologram Thickness
    delta_phase = phase % (2*math.pi)
    delta_T = delta_phase / (k_m-k_h)
    Thickness = T0 - delta_T

    alpha_t_top = (4*Z_t*(Z_h**2)*Z_m);
    alpha_t_bot_1 = (Z_h**2 * (Z_t + Z_m)**2) * pow(tf.math.cos((k_h * Thickness)), 2);
    alpha_t_bot_2 = (Z_h**2 + Z_t*Z_m)**2 * pow(tf.math.sin((k_h * Thickness)), 2);

    alpha_t = pow(alpha_t_top / (alpha_t_bot_1 + alpha_t_bot_2), 0.5)
    return alpha_t

# Calculation of PSNR ratio in dB
def calculate_psnr(original, comparison):
    M = comparison.shape[0]
    differ = np.sum(pow((original.numpy() - comparison), 2)) / (M*M)
    PSNR = 20*np.log10(255 / np.sqrt(differ))
    return PSNR

In [31]:
# Angular Spectrum Method 
# See k-Wave http://www.k-wave.org/documentation/angularSpectrumCW.php for original code
def band_limited_angular_h(k, N, l_ambda, z, p, f0, c0):
    N = N * 2
    if N%2==0:
        k_vec = np.arange(-N/2, N/2)
    else:
        k_vec = np.arange(-((N-1)/2), ((N-1)/2)+1)
    k_vec *= (2*math.pi) / (N*p)
    
    k_vec[int(N/2)] = 0
    
    k_vec = np.fft.ifftshift(k_vec)
    kx, ky = np.meshgrid(k_vec, k_vec)

    k = 2*math.pi*f0 / c0

    kz = pow(k**2 -  (pow(kx, 2) + pow(ky, 2))+0j, 0.5)
    sqrt_kx2_ky2 = pow(pow(kx,2) + pow(ky, 2), 0.5)

    H = np.conj(np.exp(1j * z * kz))
    D = (N- 1) * (p)
    kc = k * pow(0.5 * D**2 / (0.5 * D**2 + z**2) , 0.5)
    
    kh = np.where(sqrt_kx2_ky2 > kc, 0, H)
    return kh

In [32]:
### Get initialization values
N, z, p, l_ambda, T0, f0, c0 = init_param()
k = 2*np.pi/l_ambda
image_number = '5'
target_amp = 1

##### Target Amplitude #####
in_dir = './'
amp_img_np = cv2.imread(in_dir + 'test_image_' + image_number + '_amp.png', cv2.IMREAD_UNCHANGED)
size = (N, N)
amp_img_np = cv2.resize(amp_img_np, size)
original = tf.dtypes.cast(amp_img_np, tf.float64)
amp_img = tf.constant(amp_img_np/ 255.0 * target_amp)
amp_img = tf.dtypes.cast(amp_img, tf.float64)

##### Input #####
input_int = cv2.imread(in_dir + 'test_transducer.png', cv2.IMREAD_UNCHANGED)
size = (N, N)
input_int = cv2.resize(input_int, size)
input_int = input_int / 255.0
input_int = tf.constant(input_int)
input_int = tf.dtypes.cast(input_int, tf.float64)


##### Randomize phase #####
#random_phase = np.random.uniform(0.0, 2.0*np.pi, size)
zero_phase = np.zeros(size)
phase = tf.Variable(zero_phase)

##### Transducer Inputs #####
light_img = tf.dtypes.cast(input_int, tf.complex128)

# Target Image
target = amp_img

##### Get Angular Spectrum #####
angular_h = band_limited_angular_h(k, N, l_ambda, z, p, f0, c0)


In [33]:
# Calcualtes propagation of the input_image to the z-plane using angular spectrum method.
# Code was based on http://www.k-wave.org/documentation/angularSpectrumCW.php

def angular_spectrumCW(input_image, select_h):
  M = input_image.shape[0]
  init_plane_add = tf.pad(input_image, ((M//2,M//2), (M//2,M//2)), 'constant')
  N = init_plane_add.shape[0]
  fft_length = M #for case of 512 which is the case up here input size 256 x 256
  init_shift = tf.signal.fft2d(init_plane_add)
  prop = tf.signal.ifft2d(tf.math.multiply(init_shift, select_h))
  N = prop.shape[0]
  M = N // 2
  start_num = M//2
  end_num = N - M//2
  pressure = prop[start_num:end_num, start_num:end_num]

  return pressure

In [34]:
# Defining loss function 
def loss_func():
  phase_exp = tf.dtypes.complex(tf.math.cos(phase), tf.math.sin(phase))

  # Input Plane
  init_plane = tf.math.multiply(light_img, phase_exp)
  pressure = angular_spectrumCW(init_plane, angular_h)
  
  # Calculate amplitude of acoustic field
  prop_inten = pressure * tf.math.conj(pressure)
  prop_intenf = tf.dtypes.cast(prop_inten, tf.float64)
  current_phase = tf.math.angle(pressure) % (2*math.pi)
  
  if (step_count%50)==0:
    imgs = [
      amp_abs(prop_intenf.numpy()), 
      phaser_draw(current_phase)
    ]
    #show_imgs(imgs)
  loss = tf.reduce_sum(tf.math.abs(prop_intenf-target))
  #loss = tf.reduce_sum(pow((prop_intenf - target),2))
  return loss


In [35]:
# Get optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
step_count = 0;
loss_record = []
for i in range(200):
  # Optimize Function
  step_count = opt.minimize(loss_func, [phase]).numpy()

  # Evaluate performance
  ls = loss_func()
  loss_record.append(ls) 
  if (i%50) ==0:
      imgs = [
        phaser_draw(phase)
      ]
      #show_imgs(imgs)

      ls = loss_func()
      print(ls)

# Exporting Function
exporting_phase = phaser_draw(phase)
phase_exp = tf.dtypes.complex(tf.math.cos(phase), tf.math.sin(phase))
np.savetxt('test_image_' + image_number + '_loss_record_DP.csv', np.array(loss_record), delimiter=',')
cv2.imwrite('test_image_' + image_number + '_phase_ex_DP.png', exporting_phase)

tf.Tensor(33016.17727420454, shape=(), dtype=float64)
tf.Tensor(1604.8356602759386, shape=(), dtype=float64)
tf.Tensor(906.4279806617731, shape=(), dtype=float64)
tf.Tensor(840.3982358371936, shape=(), dtype=float64)


True